In [1]:
import os
import pandas as pd
from app.ml.ocr import TextRecognizer
from app.services.retriever import Retriever
import os
import cv2
import matplotlib.pyplot as plt

In [11]:
TEST_IMGS_PATH = "F:/HACKS/international/train Росатом/train//imgs"
SUBMISSION_PATH = "app/ml/metrics/submission.csv"

In [2]:
def draw_boxes(image, label):
    H, W, _ = image.shape

    for line in label.split("\n"):
        class_id, x_center, y_center, width, height = map(float, line.strip().split())

        x_min = int((x_center - width / 2) * W)
        y_min = int((y_center - height / 2) * H)
        x_max = int((x_center + width / 2) * W)
        y_max = int((y_center + height / 2) * H)

        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), thickness=5)

In [ ]:
def get_submission_csv(folder_path, output_file='submission.csv'):
    text_recognizer = TextRecognizer(['en'])
    retriever = Retriever(ngrams_count=2, local_mode=True)
    data = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
            image_path = os.path.join(folder_path, filename)
            img_name, label_text, label = text_recognizer.detect_and_read(image_path)
            retrieved_label = retriever.retrieve_most_similar_article(label_text)
            data.append({'image_file': img_name, 'label': label, 'label_text': retrieved_label})

            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            draw_boxes(image_rgb, label)

            plt.figure(figsize=(5, 5))
            plt.imshow(image_rgb)
            plt.axis('off')
            plt.show()

            if label_text:
                print(f'текст на фото : {retrieved_label}')
            else: print('текст не размечен')

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False, sep=',', encoding='utf-8')

    


In [ ]:
get_submission_csv(TEST_IMGS_PATH, SUBMISSION_PATH)

In [12]:
df = pd.read_csv(SUBMISSION_PATH, encoding='utf-8')
for index, row in df.iterrows():
    print(f"Row {index}: {row.to_dict()}")

Row 0: {'image_file': '102.JPG', 'label': '0 0.4312996031746032 0.5775462962962963 0.47668650793650796 0.7860449735449735', 'label_text': '1391-30-0108 ТС1.1'}
Row 1: {'image_file': '105.JPG', 'label': '0 0.6171875 0.5552083333333333 0.765625 0.79375', 'label_text': '1753-30-1060'}
Row 2: {'image_file': '107.JPG', 'label': '0 0.575 0.426953125 0.6 0.43984375', 'label_text': 'АМ117.06.00.002'}
Row 3: {'image_file': '109.JPG', 'label': '0 0.54375 0.5130208333333334 0.578125 0.49270833333333336', 'label_text': 'АМ117.06.01.302'}
Row 4: {'image_file': '115.JPG', 'label': '0 0.45260416666666664 0.602734375 0.5635416666666667 0.59296875', 'label_text': 'АМ116.06.00.820'}
Row 5: {'image_file': '116.JPG', 'label': '0 0.4640625 0.5046875 0.48125 0.9072916666666667', 'label_text': 'АНЕМ.715514.413'}
Row 6: {'image_file': '117.JPG', 'label': '0 0.41171875 0.49583333333333335 0.484375 0.8895833333333333', 'label_text': 'АМ116.06.00.833'}
Row 7: {'image_file': '119.JPG', 'label': '0 0.5317708333333